In [3]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [4]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
111,I bought this movie sight unseen at a sci-fi c...,negative
521,"LIGHTS OF NEW YORK was the first ""all-taking"" ...",positive
614,Jim Carrey is one of the funniest and most gif...,negative
279,"I loved this movie. To be very fair, the movie...",positive
169,While this movie did have a few scary moments ...,negative


In [5]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:32: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Manoj kumar pathak\AppData\Local\Temp\ipykernel_3576\3798096103.py:32: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [24]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to C:\Users\Manoj kumar
[nltk_data]     pathak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Manoj kumar
[nltk_data]     pathak\AppData\Roaming\nltk_data...


True

In [25]:
df = normalize_text(df)
df.head()

,review,sentiment
111,bought movie sight unseen sci fi convention go...,0
521,light new york first taking feature film comin...,1
614,jim carrey one funniest gifted comedian film t...,0
279,loved movie fair movie start slowly none chara...,1
169,movie scary moment great use music film angle ...,0


In [7]:
df['sentiment'].value_counts()

sentiment
negative    250
positive    250
Name: count, dtype: int64

In [8]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [9]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
111,i bought this movie sight unseen at a sci-fi c...,0
521,"lights of new york was the first ""all-taking"" ...",1
614,jim carrey is one of the funniest and most gif...,0
279,"i loved this movie. to be very fair, the movie...",1
169,while this movie did have a few scary moments ...,0


In [10]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [28]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [18]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/Raistar22/capstone-mlops.mlflow')
dagshub.init(repo_owner='Raistar22', repo_name='capstone-mlops', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2025-08-24 23:43:15,015 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/Raistar22/capstone-mlops "HTTP/1.1 200 OK"


Initialized MLflow to track repo "Raistar22/capstone-mlops"

2025-08-24 23:43:15,015 - INFO - Initialized MLflow to track repo "Raistar22/capstone-mlops"


Repository Raistar22/capstone-mlops initialized!

2025-08-24 23:43:15,015 - INFO - Repository Raistar22/capstone-mlops initialized!
2025/08/24 23:43:15 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/7f1cd42d9fd241be95e1de7430a79b06', creation_time=1756059201869, experiment_id='0', last_update_time=1756059201869, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [30]:
import mlflow
import logging
import os
import time
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.20)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        # ✅ Save and log the model as an artifact instead of using mlflow.sklearn.log_model
        logging.info("Saving and logging the model as artifact...")
        model_path = "logreg_model.pkl"
        joblib.dump(model, model_path)
        mlflow.log_artifact(model_path)

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Print results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-08-25 00:01:13,641 - INFO - Starting MLflow run...
2025-08-25 00:01:14,100 - INFO - Logging preprocessing parameters...
2025-08-25 00:01:15,434 - INFO - Initializing Logistic Regression model...
2025-08-25 00:01:15,434 - INFO - Fitting the model...
2025-08-25 00:01:15,470 - INFO - Model training complete.
2025-08-25 00:01:15,470 - INFO - Logging model parameters...
2025-08-25 00:01:15,815 - INFO - Making predictions...
2025-08-25 00:01:15,815 - INFO - Calculating evaluation metrics...
2025-08-25 00:01:15,825 - INFO - Logging evaluation metrics...
2025-08-25 00:01:20,759 - INFO - Saving and logging the model as artifact...
2025-08-25 00:01:22,189 - INFO - Model training and logging completed in 8.09 seconds.
2025-08-25 00:01:22,189 - INFO - Accuracy: 0.67
2025-08-25 00:01:22,189 - INFO - Precision: 0.6296296296296297
2025-08-25 00:01:22,189 - INFO - Recall: 0.723404255319149
2025-08-25 00:01:22,193 - INFO - F1 Score: 0.6732673267326733


🏃 View run magnificent-bug-686 at: https://dagshub.com/Raistar22/capstone-mlops.mlflow/#/experiments/0/runs/f39677ab1fba4b82b78cf24c0ba0176f
🧪 View experiment at: https://dagshub.com/Raistar22/capstone-mlops.mlflow/#/experiments/0
